## Costruzione del dataset di apprendimento

In [ ]:
from traceback import print_tb
import networkx as nx
import csv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

#credenziali per usare le api di spotipy
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="85aa8b30835e45a3b99623b2797df916",client_secret="ecb8aa6ad9d7431db74fb99af51dec93"))

#il file global.csv contiene i link delle classifiche top 50 di ogni paese interessato, oltre al link dela classifica top 50 global
people = list()
with open('global.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        people.append(row)

#elimino la voce corrispondente al link della playlist top 50 global
del people[0]
dataset = list()

j = 0
aux = {}
pausa = 0
pausaPaese = 0

#siccome spotify banna per 24 ore se si fanno troppe richieste in poco tempo, mi fermo per 35 secondi ogni 8 paesi considerati e, per ogni paese, mi fermo per 35 secondi ogni 8 canzoni considerate
for s in range(0,73):
    pausaPaese += 1
    if pausaPaese % 8 == 0:
        time.sleep(35)
    #per capire se sono passato al prossimo paese, questa e un'altra print mi sono utili per capire se sono stato bannato oppure no, perchè altrimenti inizia a stampare "Max Retries Reached" e non posso usare le api per 24 ore
    print('prossimo paese')
    #ricavo la playlist top 50 del paese interessato
    playlist = spotify.playlist(people[s]["Link"])
    #e ne prendo tutte le canzoni
    songs = spotify.playlist_items(playlist["id"]) 
    #considero le prime 25 canzoni (ho avuto problemi di ban anche fermandomi ogni 35 secondi, ho stabilito che 25 canzoni per paese era il numero massimo di richieste che potevo fare)
    for i in range (0,26):
        pausa += 1
        if pausa % 8 == 0:
            time.sleep(35)
        #per capire se sto scorrendo le canzoni
        print('dentro al ciclo')
        #ricavo isrc, uri della canzone e degli artisti e i generi degli artisti
        isrc = songs["items"][i]["track"]["external_ids"]["isrc"]
        track_uri = songs["items"][i]["track"]["uri"]
        artist_uri = songs["items"][i]["track"]["artists"][0]["uri"]
        genres = spotify.artist(artist_uri)["genres"]
        info = spotify.audio_features(track_uri)[0]
        
        #prelevo la danceability della canzone
        danceability = 0
        try:
            danceability = info['danceability']
        except:
            pass

        #prelevo l'energy della canzone
        energy = 0
        try:
            energy = info['energy']
        except:
            pass

        #prelevo la speechiness della canzone
        speechiness = 0
        try:
            speechiness = info['speechiness']
        except:
            pass

        #prelevo l'acousticness della canzone
        acousticness = 0
        try:
            acousticness = info['acousticness']
        except:
            pass

        #prelevo l'instrumentalness della canzone
        instrumentalness = 0
        try:
            instrumentalness = info['instrumentalness']
        except:
            pass
        
        #prelevo la valence della canzone
        valence = 0
        try:
            valence = info['valence']
        except:
            pass

        #prelevo il tempo in bpm della canzone
        tempo = 0
        try:
            tempo = info['tempo']
        except:
            pass

        #salvo queste informazioni in aux
        aux[j] = {"paese": people[s]["Name"],"isrc":isrc,"uri":track_uri,"genres":genres,"danceability":danceability,"energy":energy,"speechiness":speechiness,"acousticness":acousticness,"instrumentalness":instrumentalness,"valence":valence,"tempo":tempo}
        j = j+1
pausaPaese = 0
print('uscito dal ciclo')
print(aux)

#infine, scrivo il tutto in un file csv
with open('machine-global01.csv', 'w+', newline='') as file:
    writer = csv.DictWriter(file,fieldnames=["paese","isrc","uri","genres","danceability","energy","speechiness","acousticness","instrumentalness","valence","tempo"])
    writer.writeheader()
    for i in range (j) :
        print('scrivo il file')
        writer.writerow(aux[i])

## Applicazione della regressione logistica

In [ ]:
from datetime import datetime
from collections import Counter
from unittest import result
from joblib import PrintTime

# Data management
import pandas as pd
import csv

# Data preprocessing and trasformation (ETL)
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, FunctionTransformer, Binarizer, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml, load_iris, make_moons, make_classification
from sklearn.model_selection import train_test_split

# Math and Stat modules
import numpy as np
from scipy.stats import sem
from random import choice

# Supervised Learning
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold, StratifiedKFold, RepeatedKFold, ShuffleSplit, StratifiedShuffleSplit, learning_curve, validation_curve
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.base import BaseEstimator
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

dataset_link_prediction = pd.read_csv('machine-global-mergedfix2 - Copia.csv')

generi = set()
with open('machine-global-mergedfix2 - Copia.csv', 'r') as f:
    csv_reader = csv.reader(f)
    for line in csv_reader:
        s=line[3].replace("[","")
        s=s.replace("]","")
        s=s.replace("'","")
        genres = s.split(",")
        for genre in genres:
            if len(genre) > 0 and genre[0] == " ":
                genre = genre[1:]
            generi.add(genre)

for genere in generi:
    dataset_link_prediction[genere] = 0

#print(dataset_link_prediction)

riga = -1
with open('machine-global-mergedfix2 - Copia.csv', 'r') as f:
    csv_reader = csv.reader(f)
    for line in csv_reader:
        if line[0] != 'paese':
            riga = riga+1
            s=line[3].replace("[","")
            s=s.replace("]","")
            s=s.replace("'","")
            genres = s.split(",")
            for genre in genres:
                if len(genre) > 0 and genre[0] == " ":
                    genre = genre[1:]
                dataset_link_prediction.loc[riga,genre] = 1

X = dataset_link_prediction.iloc[:, 4:] #feature matrix
y = dataset_link_prediction['paese'] # label

#print(dataset_link_prediction.loc[1,'art rock'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


min_max_scaler = MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)

clf_logreg = LogisticRegression()
clf_logreg.fit(X_train_minmax, y_train)
X_test_minmax = min_max_scaler.transform(X_test)
y_predicted_lr = clf_logreg.predict(X_test_minmax)

print(classification_report(y_test,y_predicted_lr))

print(accuracy_score(y_test, y_predicted_lr))
#print(len(set(y_test) - set(y_predicted_lr)))








